In [1]:
from datalab.context import Context
import google.datalab.bigquery as bq
import google.datalab.storage as gcs
import numpy as np
import pandas as pd
import json
import tciaclient


In [2]:
# Basic defs

# What BQ table is being cleaned
project_id = Context.default().project_id 
source_dataset = project_id+'.working'
source_table = source_dataset+'.TCGA_radiology_images'

# What's the GQ bucket where cleaned results go 
bucket_name = project_id + '-cleaned'
object_name = 'TCGA_radiology_images'

# What's the BQ table of cleaned data
dest_dataset = project_id+'.working'
dest_table = dest_dataset+".TCGA_radiology_images_cleaned"


In [11]:
# Table we need to clean up
#table = bq.Table(source_table_name)
# Define the query tp get it
get_metadata = bq.Query("select * from `"+source_table+"` order by PatientID")
#Execute the query
query = get_metadata.execute()
# Convert to a dataframe
df = query.result().to_dataframe()
schema = query.result().schema

In [ ]:
# Replace /NA/NA with /<cancer type>/<patient id>
r = query.result()
# Build a dictionary that maps SeriesInstanceUID to cancer type
studyToCancerType = {}
# We will also build a list of lists of cleaned metadata in cleaned.
cleaned = []
# and a dictionary of old to new GCS file names
ZipFileNames = {}
# Get our TCIA api key from GCS and create a TCIAClient
tciaApiKey=gcs.Bucket(Context.default().project_id + '-dev').object('tcia_key.txt').read_stream()
# Under Python3, read_stream() returns bytes, so we have to convert to string, then split out the \n
tciaApiKey=tciaApiKey.decode().split('\n')
tcia_client = tciaclient.TCIAClient(apiKey = tciaApiKey[0],baseUrl="https://services.cancerimagingarchive.net/services/v3",resource = "TCIA")

for row in range(r.length):
  if row % 10000 == 0:
    print(row)
  #print(row)
  if r[row]["ZipFileName"].find('/NA/NA') >= 0:
    StudyInstanceUID = r[row]["StudyInstanceUID"]
    #print("Study ", StudyInstanceUID)
    if StudyInstanceUID in studyToCancerType:
      #See if we've already encountered this study ID
      cancerType = studyToCancerType[StudyInstanceUID]
      #print("Have cancer type: ", cancerType)
    else:
      #Get study metadata from TCIA
      response = tcia_client.get_series(studyInstanceUid=StudyInstanceUID)
      studyData = json.loads(response.text)
      cancerType = studyData[0]["Collection"]
      studyToCancerType[StudyInstanceUID] = cancerType
      #print("Found cancer type: ", cancerType)
    ZipFileName = r[row]["ZipFileName"]
    splitName = ZipFileName.rsplit("/",3) 
    splitName[1] = cancerType    
    splitName[2] = r[row]["PatientID"]
    newZipFileName = "/".join(splitName)
    #print(ZipFileName,"-->",r[row]["ZipFileName"])

    #See if we alreadt encountered this ZipFileName
    if not ZipFileName in ZipFileNames:
      ZipFileNames[ZipFileName] = newZipFileName
  else:
    newZipFileName = r[row]["ZipFileName"]

# Copy the data from the table row into a list
  data = [newZipFileName] 
  for element in range(1,len(schema)):
    if schema[element]['type'] == 'INTEGER' or schema[element]['type'] == 'FLOAT':
      if str(r[row][schema[element]['name']]) == 'None':
        data.append("")
        #print(row,schema[element]['name'],r[row][schema[element]['name']])
      else:
        data.append(str(r[row][schema[element]['name']]))
    else:
      data.append(r[row][schema[element]['name']])
  cleaned.append(data)
  
  


In [ ]:
# Convert the list of lists to tsv
def tabify(cleaned):
  s = str(cleaned[0])
  #print(type(s))
  for i in range(1,len(cleaned)):
    s+= "\t"
    if cleaned[i] != None:
      s+= cleaned[i]
  return s

cleaned_list = []
for clean in range(len(cleaned)):
  if clean % 10000 == 0:
    print(clean)
  cleaned_list.append(tabify(cleaned[clean]))
cleaned_tsv = "\n".join(cleaned_list)


In [64]:
bucket = gcs.Bucket(bucket_name)

# Create storage bucket if it does not exist
if not bucket.exists():
    bucket.create()
    
# Create the object 
object = bucket.object(object_name)
# Convert the dataframe to csv and write to the GCS
result = object.write_stream(content=cleaned_tsv,content_type='text/tsv')


In [65]:
# Also save the ZipFileNames dictionary
bytes = json.dumps(ZipFileNames)
zip_object = bucket.object('ZipFileNames') 
result = zip_object.write_stream(content=bytes,content_type='text/plain')


In [34]:
# Now upload the csv table in GCS to BQ
new_table = bq.Table(dest_table)
schema = query.result().schema
CSVOptions = bq.CSVOptions(delimiter="\t",skip_leading_rows=0)
new_new_table = new_table.create(schema=schema, overwrite=True)
blob = 'gs://'+bucket_name+'/'+object_name
job = new_new_table.load(blob,ignore_unknown_values=True, csv_options=CSVOptions, max_bad_records=10)

 

In [ ]:
# We need to copy files that had /NA/NA in their names to give them the correct name

dest_bucket = gcs.Bucket("cgc-05-0011-isb-tcia-open")

# Create storage bucket if it does not exist
if not dest_bucket.exists():
    dest_bucket.create()

for file in ZipFileNames:
  dst_file = ZipFileNames[file].split('/',3)
  src_file = file.split('/',3)
  source_object = gcs.Object(src_file[2], src_file[3])

  try:
    dest_object = source_object.copy_to(new_key=dst_file[3],dest_bucket)
  except Exception as e:
    print("Error copying ",file,"-->",ZipFileNames[file],": ",e)
